In [128]:
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os.path

data_sufix = 'data/'

In [129]:
# configure Session class with desired options
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')

In [130]:
hashtags = ['h1b', 'usvisa', 'immigrationreform', 'illegalImmigration', 'immigration',
'immigrantcaravan', 'caravaninvasion', 'buildthewall']
remaining_hashtags = ['illegalaliens', 'birthrightcitizenship']
hashtagsDict = {'name': hashtags}

In [131]:
hashtagData = pd.DataFrame.from_dict(hashtagsDict)
hashtagData.to_sql('hashtag', engine, if_exists='replace', index=True, index_label='id')

In [132]:
for hashtag in hashtags:
    
    hashtag_id = pd.read_sql_query('SELECT id FROM hashtag WHERE hashtag.name="'+hashtag+'"', engine).values[0][0]

    tweetData = pd.read_csv(data_sufix+hashtag+'-tweet.csv', 
                            usecols=['tweet_id','tweet_text','tweet_created_at','tweet_retweet_count',
                                     'tweet_favorite_count','user_id'])
    tweetData['hashtag_id'] = hashtag_id
    
    retweetData = pd.read_csv(data_sufix+hashtag+'-retweet.csv', 
                              usecols=['retweet_id','source_tweet_id','retweet_text','retweet_created_at','user_id'])
    
    tweetData.to_sql('tweet', engine, if_exists='append')
    retweetData.to_sql('retweet', engine, if_exists='append')

In [133]:
userData = pd.read_csv(data_sufix+'user.csv', 
                        usecols=['user_id','user_name','user_location','user_friends_count','followers_count'])

userData.to_sql('user', engine, if_exists='append')

In [134]:
# associate it with our custom Session class
Session.configure(bind=engine)